In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
df = pd.read_csv('../input/newtrain/train.csv')

In [3]:
df.head()

,pair_id,name_1,name_2,is_duplicate,Unnamed: 4
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0,NaN
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0,NaN
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0,NaN
3,4,Powermax Rubber Factory,Co. One,0,NaN
4,5,Tress A/S,Longyou Industries Park Zhejiang,0,NaN


In [4]:
df = df.drop(['Unnamed: 4'], axis = 1)

In [5]:
df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [6]:
!pip install transliterate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 203.3 kB/s eta 0:00:00


In [7]:
new = df.loc[df['name_1'].value_counts()[1060:]]

In [8]:
df.name_1.unique()

array(['Iko Industries Ltd.', 'Apcotex Industries Ltd.',
       'Rishichem Distributors Pvt., Ltd.', ..., 'Kinik Co.',
       'Pearl Patch', 'Mubea Inc.'], dtype=object)

In [9]:
new = df.name_1.unique()
new = np.append(new, df.name_2.unique())

In [10]:
new = set(new)

In [11]:
new = list(new)

In [12]:
common = [x.split(' ') for x in new]

In [13]:
def most_common(lst):
    return sorted(lst, key=lst.count, reverse = True)

In [14]:
from itertools import chain

In [15]:
comp = list(chain(*common))

In [16]:
nabor = most_common(comp)

In [17]:
comp = [x.replace('(', '') for x in comp]
comp = [x.replace(')', '') for x in comp]
comp = [x.replace('\xa0', '') for x in comp]
comp = [x.replace('"', '') for x in comp]
comp = [x for x in comp if not any([i in 'АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя' for i in x])]
comp = [x for x in comp if all([i in'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890-.@' for i in x])]

In [18]:
imena = []
for n in comp:
    if n.lower().capitalize() not in imena and len(n) > 3 and not n.isnumeric():
        imena.append(n.lower().capitalize())

In [19]:
suffix = imena[:(len(imena) // 5) + 50]
titles = imena[(len(imena) // 5) + 50:]

In [20]:
from random import choice, shuffle

In [21]:
choice(suffix)

'Basicos'

In [22]:
def make_sp():
    firstsp = []
    for tit in titles:
        firstsp.append(' '.join([tit, choice(suffix)]))
    return firstsp

In [23]:
postsuf = ['International', 
           'Inc.', 
           'Incorporation', 
           'OAO', 
           'Global', 
           'Limited', 
           'and Co.', 
           'of Moscow', 
           'of Russia', 
           'of N.Y.', 
           'Lmt.', 
           'Brotherhood', 
           'Bancgroup', 
           'Properties', 
           'Technologies', 
           'Corporation', 
           'Company', 
           'Industries', 
           'of Alabama', 
           'of Nebraska', 
           'Engineering', 
           'Energy', 
           'Entertainment', 
           'Bank', 
           'Foods', 
           'of Florida', 
           'of Hawaii', 
           'of Delaver', 
           'of Germany', 
           'of France', 
           'of Colorado']

In [24]:
def make_second_sp(first):
    secondsp = []
    for i in range(len(first)):
        var = choice(postsuf)
        while var in first[i]:
            var = choice(postsuf)
        secondsp.append(' '.join([first[i], var]))
    return secondsp
    

In [25]:
f = []
s = []
for i in range(20):
    fr = make_sp()
    sd = make_second_sp(fr)
    f = f + fr
    s = s + sd
    

In [26]:
f0 = f[1:] + f[:1]
s0 = s[::-1]

In [27]:
print(f0[:5], s0[:5])

['Social Norwegian', 'Islami Apple', 'Allnassr Dalian', 'Lalmansogat Juan', 'Rakia Capital'] ['Dhali Nations Corporation', 'Shama Body of Moscow', 'Evonic Pigipada Limited', 'Julio Wenzhou Entertainment', 'Suez Berwin Inc.']


In [28]:
dup = [1] * len(f)
dup0 = [0] * len(f0)

In [29]:
name1 = f + f0
name2 = s + s0
d = dup + dup0

In [30]:
data = pd.DataFrame(data = {'name_1':name1, 'name_2':name2, 'is_dup':d})

In [31]:
data = data[:-1]

In [32]:
data = data.sample(frac=1).reset_index(drop=True)

In [33]:
data.to_csv('data.csv', index = False)